# Generalized Linear Models

This notebook introduces the algorithms within [Dask-GLM](https://github.com/dask/dask-glm) for [Generalized Linear Models](https://en.wikipedia.org/wiki/Generalized_linear_model).

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [13]:
from dask.distributed import Client, progress
client = Client(processes=False, threads_per_worker=4,
                n_workers=1, memory_limit='2GB')
client

/opt/conda/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: inproc://172.31.33.28/32220/10 Dashboard: http://localhost:40542/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


In [ ]:
!pip install --user dask_glm

## Make a random dataset

In [4]:
from dask_glm.datasets import make_regression
X, y = make_regression(n_samples=200000, n_features=100, n_informative=5, chunksize=10000)
X

dask.array<normal, shape=(200000, 100), dtype=float64, chunksize=(10000, 100), chunktype=numpy.ndarray>

In [5]:
import dask
X, y = dask.persist(X, y)

## Solve with a GLM algorithm

*We also recommend looking at the "Graph" dashboard during execution if available*

In [6]:
import dask_glm.algorithms

b = dask_glm.algorithms.admm(X, y, max_iter=5)

## Solve with a difference GLM algorithm

In [7]:
b = dask_glm.algorithms.proximal_grad(X, y, max_iter=5)

/opt/conda/lib/python3.7/site-packages/dask/core.py:119: RuntimeWarning: overflow encountered in exp
  return func(*args2)


## Customizable with different families and regularizers

The Dask-GLM project is nicely modular, allowing for different GLM families and regularizers, including a relatively straightforward interface for implementing custom ones.

In [8]:
import dask_glm.families
import dask_glm.regularizers

family = dask_glm.families.Poisson()
regularizer = dask_glm.regularizers.ElasticNet()

b = dask_glm.algorithms.proximal_grad(
    X, y, 
    max_iter=5, 
    family=family,
    regularizer=regularizer,
)

/opt/conda/lib/python3.7/site-packages/dask/core.py:119: RuntimeWarning: overflow encountered in exp
  return func(*args2)
/opt/conda/lib/python3.7/site-packages/dask/core.py:119: RuntimeWarning: overflow encountered in exp
  return func(*args2)
/opt/conda/lib/python3.7/site-packages/dask/core.py:119: RuntimeWarning: overflow encountered in exp
  return func(*args2)


In [9]:
dask_glm.families.Poisson??

Init signature: dask_glm.families.Poisson()
Source:        
class Poisson(object):
    """
    This implements `Poisson regression`_

    Useful for modelling count data.

    .. _Poisson regression: https://en.wikipedia.org/wiki/Poisson_regression
    """
    @staticmethod
    def loglike(Xbeta, y):
        eXbeta = exp(Xbeta)
        yXbeta = y * Xbeta
        return (eXbeta - yXbeta).sum()

    @staticmethod
    def pointwise_loss(beta, X, y):
        beta, y = beta.ravel(), y.ravel()
        Xbeta = X.dot(beta)
        return Poisson.loglike(Xbeta, y)

    @staticmethod
    def pointwise_gradient(beta, X, y):
        beta, y = beta.ravel(), y.ravel()
        Xbeta = X.dot(beta)
        return Poisson.gradient(Xbeta, X, y)

    @staticmethod
    def gradient(Xbeta, X, y):
        eXbeta = exp(Xbeta)
        return dot(X.T, eXbeta - y)

    @staticmethod
    def hessian(Xbeta, X):
        eXbeta = exp(Xbeta)
        x_diag_eXbeta = eXbeta[:, None] * X
        return dot(X.T, x_diag_e

In [10]:
dask_glm.regularizers.ElasticNet??

Init signature: dask_glm.regularizers.ElasticNet(weight=0.5)
Source:        
class ElasticNet(Regularizer):
    """Elastic net regularization."""
    name = 'elastic_net'

    def __init__(self, weight=0.5):
        self.weight = weight
        self.l1 = L1()
        self.l2 = L2()

    def _weighted(self, left, right):
        return self.weight * left + (1 - self.weight) * right

    def f(self, beta):
        return self._weighted(self.l1.f(beta), self.l2.f(beta))

    def gradient(self, beta):
        return self._weighted(self.l1.gradient(beta), self.l2.gradient(beta))

    def hessian(self, beta):
        return self._weighted(self.l1.hessian(beta), self.l2.hessian(beta))

    def proximal_operator(self, beta, t):
        """See notebooks/ElasticNetProximalOperatorDerivation.ipynb for derivation."""
        g = self.weight * t

        @np.vectorize
        def func(b):
            if b <= g:
                return 0
            return (b - g * np.sign(b)) / (t - g + 1)
        r

In [11]:
client.close()

In [12]:
%reset -s -f